# Libraries

In [92]:
import math
import random
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# Module 1 + Module 2
By combining *Module 1* and *Module 2* the problem is formally defined as follows:
- a depot $0$ where all vehicles are located in the beginning
- a set of customers $C = {1,...,N}$, we define locations $L = {0} + C$
- a demand $d_i$ for each customer $i \in C$ (the depot has no demand)
- a time window $[a_i, b_i]$ with $0<a_i<b_i$ for each location $i \in L$ (including the depot)
- a service time $s_i$ for each customer $i \in C$ (the depot has zero service time)
- costs $c_{ij}$ and time $t_{ij}$ for travelling from location $i$ to location $j$
- a homogeneous fleet $K = {1,...,|K|}$ of vehicles with load capacity $Q$

The **objective** is:
- to find a route for each vehicle, i.e., an ordered sequence of customers
- with minimal total routing time, such that:
    - each customer is visited exactly once
    - the start times of the services have to be within the depot and customer time windows (potentially introducing waiting times between services),
    - and the vehicle capacity is never exceeded

## Input Configuration
We configurate the input values:

In [93]:
numCustomers = 5
maxNumVehicles = 2

vehicleCapacity = 30
demandRange = (2, 5)

timeHorizon = 100 # tmax
timeWindowWidthRange = (10, 15)
serviceTimeRange = (2, 4)

## Instance Generation
By using the values in the *Input Configuration*, we generate the instances of the problem:

In [94]:
random.seed(0)

depot = 0
customers = [*range(1, numCustomers + 1)] # array of customers
locations = [depot] + customers # depot + customers
connections = [(i, j) for i in locations for j in locations if i != j] # set of all possible edges
vehicles = [*range(1, maxNumVehicles + 1)] # array of vehicles

# create random depot and customer locations in the Euclidian plane (1000x1000)
points = [(random.randint(0, 999), random.randint(0, 999)) for i in locations]
# dictionary of Euclidean distance for each connection (interpreted as travel costs)
costs = {
    (i, j): math.ceil(
        math.sqrt(sum((points[i][k] - points[j][k]) ** 2 for k in range(2)))
    )
    for (i, j) in connections
}
maximalCosts = math.ceil(999 * math.sqrt(2))

# dictionary of travel times for each connection (related to the costs, scaled to time horizon)
travelTimes = {
    (i, j): math.ceil((costs[i, j] / maximalCosts) * timeHorizon * 0.2)
    for (i, j) in connections
}

# create random demands
demands = {i: random.randint(demandRange[0], demandRange[1]) for i in customers}
demands[0] = 0  # depot has no demand

# create random service times,
serviceTimes = {
    i: random.randint(serviceTimeRange[0], serviceTimeRange[1]) for i in customers
}
serviceTimes[0] = 0  # depot has no service time

# create random time windows
timeWindowWidths = {
    i: random.randint(timeWindowWidthRange[0], timeWindowWidthRange[1])
    for i in customers
}

# vehicles are allowed to leave the depot any time within the time horizon
timeWindowWidths[0] = timeHorizon

# create time windows randomly based on the previously generated information
# such that the service at a customer can be finished within the time horizon
timeWindows = {}
timeWindows[0] = (0, 0 + timeWindowWidths[0])
for i in customers:
    start = random.randint(0, timeHorizon - serviceTimes[i] - timeWindowWidths[i] - travelTimes[i,0])
    timeWindows[i] = (start, start + timeWindowWidths[i])

So now, we can print all these parameters:

In [95]:
print("Customer Demands:")
for customer, demand in demands.items():
    print(f"Customer {customer}: Demand = {demand}")

print("\nCustomer Service Times:")
for customer, service_time in serviceTimes.items():
    print(f"Customer {customer}: Service Time = {service_time}")

print("\nCustomer Time Windows:")
for customer, window in timeWindows.items():
    print(f"Customer {customer}: Time Window = {window}")

print("\nTravel Times:")
for (i, j), time in list(travelTimes.items()):
    print(f"Travel Time from {i} to {j}: {time}")

Customer Demands:
Customer 1: Demand = 4
Customer 2: Demand = 5
Customer 3: Demand = 4
Customer 4: Demand = 3
Customer 5: Demand = 3
Customer 0: Demand = 0

Customer Service Times:
Customer 1: Service Time = 3
Customer 2: Service Time = 2
Customer 3: Service Time = 2
Customer 4: Service Time = 4
Customer 5: Service Time = 3
Customer 0: Service Time = 0

Customer Time Windows:
Customer 0: Time Window = (0, 100)
Customer 1: Time Window = (12, 26)
Customer 2: Time Window = (9, 24)
Customer 3: Time Window = (42, 56)
Customer 4: Time Window = (60, 71)
Customer 5: Time Window = (71, 83)

Travel Times:
Travel Time from 0 to 1: 8
Travel Time from 0 to 2: 8
Travel Time from 0 to 3: 12
Travel Time from 0 to 4: 6
Travel Time from 0 to 5: 11
Travel Time from 1 to 0: 8
Travel Time from 1 to 2: 14
Travel Time from 1 to 3: 8
Travel Time from 1 to 4: 7
Travel Time from 1 to 5: 6
Travel Time from 2 to 0: 8
Travel Time from 2 to 1: 14
Travel Time from 2 to 3: 14
Travel Time from 2 to 4: 7
Travel Time fr

## Model Generation
In this section we will see two different mathematical formulations of this problem:
- Big - M Model
- Flow Model

In particular, the two models just mentioned share the same basic model (Vehicle Routing Problem), but differ with regard to capacity and time constraints.

### Big - M Model
**Decision Variables**:
- $x_{ij} = 
\begin{cases} 
1 & \text{if a vehicle traverses arc (i, j)} \\
0 & \text{otherwise}
\end{cases}
$
- $y_i$ for each location $i \in L$ to denote the **vehicle load after picking up the demand at $i$**.
- $z_i$ for each location $i \in L$ to denote the **start time of the service at $i$** that has to be within its time window

In [96]:
model_M = gp.Model("VRPTW")

x = model_M.addVars(connections, vtype=GRB.BINARY, name="x")

y = model_M.addVars(locations, lb=0, ub=vehicleCapacity, name="y")
y[0].UB = 0

z = model_M.addVars(locations, name="z")
for i in locations:
    z[i].LB = timeWindows[i][0] 
    z[i].UB = timeWindows[i][1]

**Objective Function:**

Minimize the total travel time. Travel time doesn't consider the service time of each customer:
$$
\text{min} \sum_{i,j \in \mathcal{L}} c_{ij}x_{ij}
$$

In [97]:
model_M.setObjective(gp.quicksum(travelTimes[i,j]*x[i,j] for i,j in connections), GRB.MINIMIZE)

**Constraints:**
- Each customer is visited exactly once, i.e., there is exactly one incoming connection and exactly one    outgoing connection:
$$
\sum_{j \in \mathcal{L}} x_{ij} = 1 \quad \forall i \in \mathcal{C}
$$

$$
\sum_{i \in \mathcal{L}} x_{ij} = 1 \quad \forall j \in \mathcal{C}
$$

In [98]:
model_M.addConstrs((x.sum("*", j) == 1 for j in customers), name="incoming")
model_M.addConstrs((x.sum(i, "*") == 1 for i in customers), name="outgoing")

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

- At most $|K|$ vehicles can be used:
$$
\sum_{j \in \mathcal{C}} x_{0j} \leq |K|
$$

In [99]:
model_M.addConstr(x.sum(0, "*") <= maxNumVehicles, name="maxNumVehicles")

<gurobi.Constr *Awaiting Model Update*>

- We can also compute a lower bound on the number of needed vehicles, that is the sum of all demands divided by the vehicle capacity:
$$
\sum_{j \in \mathcal{C}} x_{0j} \geq \left\lceil \frac{\sum_{i \in \mathcal{C}} d_i}{Q} \right\rceil
$$

In [100]:
def numVehiclesNeededForCustomers(customers):
    sumDemand = 0
    for i in customers:
        sumDemand += demands[i]
    return math.ceil(sumDemand / vehicleCapacity)

model_M.addConstr(
    x.sum(0, "*") >= numVehiclesNeededForCustomers(customers),
    name="minNumVehicles",
)

<gurobi.Constr *Awaiting Model Update*>

- The current capacity of the vehicle must not exceed the maximum capacity of the vehicle:
    $$y_i + d_j \le y_j + Q(1 - x_{ij}) \quad \forall i \in L, j \in C, i \neq j$$
    In particular:
    - if $x_{ij} = 1$: the vehicle travels from $i$ to $j$. So, $y_j$ must be equal or greater than $y_i + d_j$
    - if $x_{ij} = 0$: it ensures that vehicle capacity si not exceed.

In [101]:
model_M.addConstrs(
    (
        y[i] + demands[j] <= y[j] + vehicleCapacity * (1 - x[i, j])
        for i in locations
        for j in customers
        if i != j
    ),
    name="loadBigM1",
)

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,


- This constraint ensures that, if arc $(i, j)$ is travelled then the load at node $j$ must be greater or equal to $y_i + d_j$:
    $$y_i + d_j \geq y_j - M_{ij}(1 - x_{ij}) \quad \forall i \in L, j \in C, i \neq j$$
    In particular, if the arc $(i, j)$ is not travelled then the inequality becomes non-binding since $M$ is a big constant

In [102]:
model_M.addConstrs(
    (
        y[i] + demands[j]
        >= y[j] - (vehicleCapacity - demands[i] - demands[j]) * (1 - x[i, j])
        for i in locations
        for j in customers
        if i != j
    ),
    name="loadBigM2",
)

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,


- This constraint manages the time required to pass from a node $i$ to a node $j$, taking into account the service time at node $i$ ($s(i)$) and the travel time between $i$ and $j$ ($t_{ij}$):
    $$z_i + s_i + t_{ij} \leq z_j + T(1 - x_{ij}) \quad \forall i \in L, j \in C, i \neq j$$
    In particular, if the arc $(i, j)$ is not travelled, then $x_{ij} = 0$ and the constraint is not activated because of the big term $T$.

In [103]:
model_M.addConstrs(
        (
            z[i] + serviceTimes[i] + travelTimes[i, j]
            <= z[j]
            + (
                timeWindows[i][1]
                + serviceTimes[i]
                + travelTimes[i, j]
                - timeWindows[j][0]
            )
            * (1 - x[i, j])
            for i in locations
            for j in customers
            if i != j
        ),
        name="timeBigM",
    )

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,


So, we can now optimize the model and plot the solution:

In [104]:
model_M.Params.OutputFlag = 0

model_M.optimize()

if model_M.SolCount >= 1:

    print("Objective Value: ", model_M.objVal)

    usedConnections = [(i, j) for (i, j) in x.keys() if x[i, j].X > 0.5]

    # create a dict for the next customer based on the current one
    # (note that the depot in general has multiple outgoing connections)
    nextCustomer = {}
    for (i, j) in usedConnections:
        if i == 0:
            if 0 not in nextCustomer.keys():
                nextCustomer[0] = []
            nextCustomer[0].append(j)
        else:
            nextCustomer[i] = j

    print(f"Solution contains {len(nextCustomer[0])} routes:")
    routeNumber = 0
    visitedCustomers = [False] * (numCustomers + 1)
    for firstCustomer in nextCustomer[0]:
        print(f"Route {routeNumber}: 0 -> ", end="")
        vehicleLoad = 0
        time = travelTimes[0, firstCustomer]
        violatedTimeWindows = False
        currentCustomer = firstCustomer
        while currentCustomer != 0:
            print(f"{currentCustomer} (L:{vehicleLoad}, T:{time}) -> ", end="")
            visitedCustomers[currentCustomer] = True
            vehicleLoad += demands[currentCustomer]
            time = max(time, timeWindows[currentCustomer][0])
            if time > timeWindows[currentCustomer][1]:
                violatedTimeWindows = True
            time += (
                serviceTimes[currentCustomer]
                + travelTimes[currentCustomer, nextCustomer[currentCustomer]]
            )
            currentCustomer = nextCustomer[currentCustomer]
        print(f"0 (L:{vehicleLoad}/{vehicleCapacity}, T:{time})")
        if vehicleLoad > vehicleCapacity:
            print("Vehicle capacity is exceeded!")
        if violatedTimeWindows:
            print("Time windows are violated!")
        routeNumber += 1

    print("Unvisited customers: ", end="")
    for c in customers:
        if visitedCustomers[c] == False:
            print(f"{c}, ", end="")
    print()
    for c in customers: 
        print(f"Client {c}: service start time = {model_M.getAttr('x', z)[c]}, time window = {timeWindows[c]}")

Objective Value:  51.0
Solution contains 2 routes:
Route 0: 0 -> 1 (L:0, T:8) -> 3 (L:4, T:23) -> 5 (L:8, T:47) -> 0 (L:11/30, T:85)
Route 1: 0 -> 2 (L:0, T:8) -> 4 (L:5, T:18) -> 0 (L:8/30, T:70)
Unvisited customers: 
Client 1: service start time = 26.0, time window = (12, 26)
Client 2: service start time = 9.0, time window = (9, 24)
Client 3: service start time = 42.0, time window = (42, 56)
Client 4: service start time = 60.0, time window = (60, 71)
Client 5: service start time = 83.0, time window = (71, 83)


### Flow Model
Now, we extend the meaning of the variables used in the Big-M model. We do not associate resource states to customers anymore but instead to connections between customers.

**Decision Variables:**
- $x_{ij} = 
\begin{cases} 
1 & \text{if a vehicle traverses arc (i, j)} \\
0 & \text{otherwise}
\end{cases}
$
- $y_{ij}$ for each connection $(i, j)$ to denote the vehicle load after picking up the demand at $i$ and proceeding to location $j$.
- $z_{ij}$ for each connection $(i, j)$ to denote the start time of the service at $i$ (which must be within its time window) when immediately proceeding to location $j$.



In [105]:
model_flow = gp.Model("VRPTW")

x = model_flow.addVars(connections, vtype=GRB.BINARY, name="x")

y = model_flow.addVars(connections, lb=0, ub=vehicleCapacity, name="y")

for i in customers:
    y[0, i].UB = 0

z = model_flow.addVars(connections, lb=0, name="z")

for (i, j) in connections:
    z[i, j].UB = timeWindows[i][1]

**Objective Function:**

Minimize the total travel time. Travel time doesn't consider the service time of each customer:
$$
\text{min} \sum_{i,j \in \mathcal{L}} c_{ij}x_{ij}
$$

In [106]:
model_flow.setObjective(gp.quicksum(travelTimes[i,j]*x[i,j] for i,j in connections), GRB.MINIMIZE)

**Constraints:**